# Using intake to access the Australian Community Reference  Climate Data Collection at NCI 

This notebook will show you how you can use the intake catalogue we prepared for the aus-ref-clim-data-nci collection hosted ont he NCI server under the `ia39` project.<br>
To access the data and the catalogue you need to part of the ia39 project<br><br>

You also need to have intake and intake-esm installed.<br>
NB as long as intake-esm is installed you only need to load intake in your code.

In [1]:
import intake

In [20]:
cat = intake.open_catalog('/g/data/ia39/aus-ref-clim-data-nci/acs-replica-intake/catalogue.yaml')
list(cat)

['cmip6_etccdi', 'gpcc', 'gpcp', 'cmap', 'frogs', 'ghcn', 'cmorph']

If you are using the CLEX managed hh5 conda environments, both modules are already installed together with the catalogue. So you can use the shortcut below to load the catalogue.

In [ ]:
#!module use /g/data3/hh5/public/modules 
#!module load conda/analysis3
# cat = intake.cat.acsreplica

NB you can also use `cat._entries` to see a much more detailed description of catalogue entries.

In [3]:
# NB you can also use cat._entries to see a much more detailed description of catalogue entries
cat._entries

{'cmip6_etccdi': name: cmip6_etccdi
 container: xarray
 plugin: ['esm_datastore']
 driver: ['esm_datastore']
 description: Replica of the Climate extreme indices and heat stress indicators derived from CMIP6 global climate projections dataset (CICERO_ETCCDI) from the Copernicus Climate Datastore on gadi. This collection is a copy of CICERO_ETCCDI, which includes climate indices calculated on CMIP6 models historical and projections experiments. The collection includes ETCCDI (climate extrems indices) at yearly and monthly resolution, and HSI (Heat Stress Indicators) at daily resolution.
 This dataset is part of the Australian Community Reference Climate Data Collection at NCI.
 The files were downloaded from the CDS as netcdf files and compressed using nccopy.
 
 direct_access: forbid
 user_parameters: []
 metadata: 
 args: 
   esmcol_obj: {{CATALOG_DIR}}/cmip6_etccdi/catalogue.json,
 'gpcc': name: gpcc
 container: xarray
 plugin: ['esm_datastore']
 driver: ['esm_datastore']
 descriptio

In [4]:
# load the entry for cmip6_etccdi, you can see etcddi is a pandas DataFrame
etccdi = cat['cmip6_etccdi']
etccdi

,unique
path,33220
index_type,2
base,5
frequency,3
experiment,5
model,27
ensemble,77
variable,32
date_range,22


You can see `etcddi` is a pandas DataFrame, with `path` and all the dataset attributes as columns.<br>
As it is a DataFrame you can use pandas methods like `head` and `columns` via the `df` accessor.

In [5]:
print(f"Columns are {etccdi.df.columns}\n")
etccdi.df.head()

Columns are Index(['path', 'index_type', 'base', 'frequency', 'experiment', 'model',
       'ensemble', 'variable', 'date_range'],
      dtype='object')



,path,index_type,base,frequency,experiment,model,ensemble,variable,date_range
0,/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccd...,etccdi,base_independent,yr,ssp370,ACCESS-CM2,r1i1p1f1,txx,2015-2100
1,/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccd...,etccdi,base_independent,yr,ssp370,ACCESS-CM2,r1i1p1f1,r10mm,2015-2100
2,/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccd...,etccdi,base_independent,yr,ssp370,ACCESS-CM2,r1i1p1f1,tnn,2015-2100
3,/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccd...,etccdi,base_independent,yr,ssp370,ACCESS-CM2,r1i1p1f1,tnx,2015-2100
4,/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccd...,etccdi,base_independent,yr,ssp370,ACCESS-CM2,r1i1p1f1,txn,2015-2100


In [ ]:
# to see all the methods are available for the catalogue use dir()
# dir(etccdi)

Let's check what is in `etccdi`, we can use:<br>
 * `description` to get an overview;
 * `aggregation_info` to check aggreagtion options for this dataset
 * `unique()` to return all the unique values available for each column
 
As unique() is dictionary if we know the columns we can directly access the values for a selected key, as shown below.

In [6]:
print(etccdi.description)
print()
print(etccdi.aggregation_info)
print()
etccdi.unique()['frequency']

Replica of the Climate extreme indices and heat stress indicators derived from CMIP6 global climate projections dataset (CICERO_ETCCDI) from the Copernicus Climate Datastore on gadi. This collection is a copy of CICERO_ETCCDI, which includes climate indices calculated on CMIP6 models historical and projections experiments. The collection includes ETCCDI (climate extrems indices) at yearly and monthly resolution, and HSI (Heat Stress Indicators) at daily resolution.
This dataset is part of the Australian Community Reference Climate Data Collection at NCI.
The files were downloaded from the CDS as netcdf files and compressed using nccopy.


AggregationInfo(groupby_attrs=['index_type', 'base', 'frequency', 'experiment', 'model', 'ensemble'], variable_column_name='variable', aggregations=[{'type': 'union', 'attribute_name': 'variable'}], agg_columns=['variable'], aggregation_dict={'variable': {'type': 'union'}})



{'count': 3, 'values': ['day', 'yr', 'mon']}

We can execute a query using the method `search()`. Let's select a subset passing the search() method some constraints.

In [7]:
subset = etccdi.search( base='base_independent', 
                        frequency='mon', 
                        model='ACCESS-CM2', 
                        experiment='historical', 
                        ensemble='r1i1p1f1' )
subset

,unique
path,7
index_type,1
base,1
frequency,1
experiment,1
model,1
ensemble,1
variable,7
date_range,1


Our subset consists of 7 files representing 7 variables for the same model, ensemble, experiment, etc.<br>
To actually load the data in xarray, we first have to create a dictionary of datasets using `to_dataset_dict()`.

In [8]:
dset_dict = subset.to_dataset_dict()
dset_dict.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'index_type.base.frequency.experiment.model.ensemble'


dict_keys(['etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1'])

Finally we can simply load a dataset using its key

In [9]:
ds = dset_dict['etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1']
ds

<xarray.Dataset>
Dimensions:       (lon: 192, bnds: 2, lat: 144, time: 1980)
Coordinates:
  * lon           (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
  * lat           (lat) float64 -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
    height        float64 9.969e+36
  * time          (time) datetime64[ns] 1850-01-16 ... 2014-12-16
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lat_bnds      (lat, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
    txxETCCDI     (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    tnxETCCDI     (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    rx1dayETCCDI  (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    tnnETCCDI     (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    dtrETCCDI     (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    txnETCCDI     (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
    rx5dayETCCDI  (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
Attributes: (12/58)
    variable_id:              tasmax
    institution:              CSIRO (Commonwealth Scientific and Industrial R...
    parent_mip_era:           CMIP6
    input_frequency:          day
    ETCCDI_software_version:  1.1.9.1
    cmor_version:             3.4.0
    ...                       ...
    input_tracking_id:        hdl:21.14100/9273db06-8b75-4cd5-a2de-7d8dd6601bf7
    ETCCDI_institution:       Center for International Climate and Environmen...
    parent_experiment_id:     piControl
    CDI:                      Climate Data Interface version 1.8.0 (http://mp...
    variant_label:            r1i1p1f1
    intake_esm_dataset_key:   etccdi.base_independent.mon.historical.ACCESS-C...

Please note that the aggregation define for this dataset has been automatically applied, so the 7 variables files have been united in one xarray Dataset.<br>
If you want to access the files without aggregation you set the optional argiment `aggregate` to False.<br>

In [10]:
dset_dict2 = subset.to_dataset_dict(aggregate=False)
dset_dict2.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'path.index_type.base.frequency.experiment.model.ensemble.variable.date_range'


dict_keys(['/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/ACCESS-CM2/rx5dayETCCDI_mon_ACCESS-CM2_historical_r1i1p1f1_no-base_v20191108_185001-201412_v1-0.nc.etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1.rx5day.185001-201412', '/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/ACCESS-CM2/txxETCCDI_mon_ACCESS-CM2_historical_r1i1p1f1_no-base_v20191108_185001-201412_v1-0.nc.etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1.txx.185001-201412', '/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/ACCESS-CM2/rx1dayETCCDI_mon_ACCESS-CM2_historical_r1i1p1f1_no-base_v20191108_185001-201412_v1-0.nc.etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1.rx1day.185001-201412', '/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/ACCESS-CM2/txnETCCDI_mon_ACCESS-CM2_historical_r1i1p1f1_no-b

This time we can a key per file, and the key is the file path.<br>
We can then load one of the file as we've done before using its key/path.

In [11]:
# Finally we can simply load a dataset using its key
ds = dset_dict2['/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/ACCESS-CM2/rx5dayETCCDI_mon_ACCESS-CM2_historical_r1i1p1f1_no-base_v20191108_185001-201412_v1-0.nc.etccdi.base_independent.mon.historical.ACCESS-CM2.r1i1p1f1.rx5day.185001-201412']
ds

<xarray.Dataset>
Dimensions:       (lon: 192, bnds: 2, lat: 144, time: 1980)
Coordinates:
  * lon           (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
  * lat           (lat) float64 -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
    height        float64 ...
  * time          (time) datetime64[ns] 1850-01-16 ... 2014-12-16
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lat_bnds      (lat, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
    rx5dayETCCDI  (time, lat, lon) float32 dask.array<chunksize=(1980, 144, 192), meta=np.ndarray>
Attributes: (12/58)
    CDI:                      Climate Data Interface version 1.8.0 (http://mp...
    history:                  Fri Jan 31 11:18:16 2020: cdo mergetime tasmax_...
    source:                   ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\...
    institution:              CSIRO (Commonwealth Scientific and Industrial R...
    Conventions:              CF-1.7 CMIP-6.2
    activity_id:              CMIP
    ...                       ...
    ETCCDI_software_version:  1.1.9.1
    frequency:                mon
    creation_date:            2020-01-31T14:29:22Z
    title:                    ETCCDI indices computed on ACCESS-CM2 output pr...
    intake_esm_varname:       None
    intake_esm_dataset_key:   /g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi...

## Other datasets 

### GPCP aggregation along time dimension

With the `cmip6_etccdi` dataset we used an aggregation of type `union`. We can also aggregate files along the time dimension. We will use the `gpcp` dataset to demonstrate.<br>
The `gpcp` daily data is organised as *frequency/version/year/files*.

In [12]:
gpcp = cat['gpcp']
gpcp.unique().keys()

dict_keys(['path', 'frequency', 'version', 'year', 'date', 'variable'])

Let's get all the daily files for the year 1999.

In [13]:
subset = gpcp.search(frequency='day', year='1999')

This subset contains 365 files and we want to load them as one aggregated dataset.

In [14]:
ds_dict = subset.to_dataset_dict()
ds_dict.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'version.frequency'


dict_keys(['v1-3.day'])

As we can see from the keys all files are united in one aggregation, that we can load as a xarray Dataset.

In [15]:
gpcc_ds = ds_dict['v1-3.day']
gpcc_ds

<xarray.Dataset>
Dimensions:      (latitude: 180, longitude: 360, time: 365, nv: 2)
Coordinates:
  * latitude     (latitude) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0
  * longitude    (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * time         (time) datetime64[ns] 1999-01-01 1999-01-02 ... 1999-12-31
Dimensions without coordinates: nv
Data variables:
    lat_bounds   (latitude, nv) float32 -90.0 -89.0 -89.0 ... 89.0 89.0 90.0
    lon_bounds   (longitude, nv) float32 0.0 1.0 1.0 2.0 ... 359.0 359.0 360.0
    time_bounds  (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    precip       (time, latitude, longitude) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes: (12/47)
    standard_name_vocabulary:   CF Standard Name Table (v41, 22 February 2017)
    institution:                ACADEMIC > UMD/ESSIC > Earth System Science I...
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    geospatial_lat_resolution:  1 degree
    comment:                    Processing computer: eagle2.umd.edu
    summary:                    Global Precipitation Climatology Project (GPC...
    ...                         ...
    intake_esm_varname:         ['precip']
    history:                    1) 2017-05-30T16:56:13Z, Dr. Jian-Jian Wang, ...
    spatial_resolution:         1 degree
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ea...
    acknowledgment:             This project was supported in part by a grant...
    intake_esm_dataset_key:     v1-3.day

### CMORPH selecting multiple years

In [27]:
cmorph = cat.cmorph
cmorph.df.columns

Index(['path', 'version', 'frequency', 'grid', 'year', 'month', 'date',
       'variable'],
      dtype='object')

CMORPH has a 'year' column and we can use it to select multiple years. To do so we pass a list of years to `search()`. In the example below range() is used to write a list including years from 2003 to 2005.

In [28]:
cmorph_sub = cmorph.search(year=[x for x in range(2003,2006)])
cmorph_sub

,unique
path,1096
version,1
frequency,1
grid,1
year,3
month,12
date,1096
variable,1


In [22]:
cmorph_sub = cmorph.search(year=[x for x in range(2003,2006)])

As these are 1096 files the aggregation is a bit slower, intake will show us a progress bar.

In [23]:
cmorph_dict = cmorph_sub.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'frequency.grid'


In [24]:
cmorph_dict.keys()

dict_keys(['30min.8km'])

In [29]:
ds = cmorph_dict['30min.8km']
ds

<xarray.Dataset>
Dimensions:      (time: 2192, nv: 2, lat: 1649, lon: 4948)
Coordinates:
  * time         (time) datetime64[ns] 2003-01-01 ... 2005-12-31T00:30:00
  * lat          (lat) float64 -59.96 -59.89 -59.82 -59.75 ... 59.82 59.89 59.96
  * lon          (lon) float64 0.03638 0.1091 0.1819 ... 359.8 359.9 360.0
Dimensions without coordinates: nv
Data variables:
    time_bounds  (time, nv) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
    lat_bounds   (lat, nv) float64 -60.0 -59.93 -59.93 ... 59.93 59.93 60.0
    lon_bounds   (lon, nv) float64 0.0 0.07276 0.07276 ... 359.9 359.9 360.0
    cmorph       (time, lat, lon) float32 dask.array<chunksize=(2, 1649, 4948), meta=np.ndarray>
Attributes: (12/55)
    standard_name_vocabulary:   CF Standard Name Table (v47, 19 September 2017)
    institution:                DOC/NOAA/NWS/NCEP/CPC > Climate Prediction Ce...
    instrument_vocabulary:      NASA Global Change Master Directory (GCMD) In...
    creator_institution:        DOC/NOAA/NWS/NCEP/CPC > Climate Prediction Ce...
    geospatial_lat_resolution:  0.072771376
    comment:                    Please contact Pingping Xie (pingping.xie@noa...
    ...                         ...
    instrument:                 SSM/I, AMSU-B, AMSR-E, TMI, MHS, MWRI
    spatial_resolution:         0.0728x0.0728 deg lat/lon (~8km at the Equator)
    ncei_template_version:      NCEI_NetCDF_Grid_template_V2.0
    creator_url:                http://www.cpc.noaa.gov/
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ea...
    intake_esm_dataset_key:     30min.8km

### GHCN csv files

The GHNC dataset is composed by csv files. We included in the catalogue, so users can see they also exists, but we are still working on their configuration, so they can be also loaded using the intake-csv driver.

In [37]:
ghcn = cat.ghcn
ghcn

ghcn:
  args:
    urlpath: /g/data/ia39/aus-ref-clim-data-nci/ghcn/data/daily/by_year/{year}.csv
  description: 'Replica of the daily station precipitation data from the Global Historical
    Climatology Network (GHCN).

    GHCN-Daily is an integrated database of daily climate summaries from land surface
    stations across the globe.

    It contains records from over 100,000 stations in 180 countries and territories.
    The period of record station files are parsed into yearly csv files that contain
    all available GHCN Daily station data for that year plus a time of observation
    field (where available).


    !!!! NB. This is a work in progress! Currently the intake-csv driver is not handling
    correctly these files.

    '
  driver: intake.source.csv.CSVSource
  metadata:
    catalog_dir: /g/data/ia39/aus-ref-clim-data-nci/acs-replica-intake/


As this is using the `csv` package to handle the data, we need first to read the data as we would for a csv file.

In [ ]:
# This is still not working probably the type of data, as some are strings!
# ghcn.read()